# Практическое задание 3
# Генерация bash команды по текстовому запросу
## курс "Математические методы анализа текстов"
### ФИО:

### Постановка задачи

В этом задании вы построите систему, выдающую пользователю последовательность утилит командной строки linux (с нужными флагами) по его текстовому запросу. Вам дан набор пар текстовый запрос - команда на выходе.

Решение этого задания будет построено на encoder-decoder архитектуре и модели transformer.


### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
* pytorch
* transformers
* sentencepiece (bpe токенизация)
* clai utils (скачать с гитхаба отсюда https://github.com/IBM/clai/tree/nlc2cmd/utils)


### Данные

В качестве обучающей выборке используются данные, сгенерированные автоматически по запросам с сайта stack overflow. В качестве тестовых данных используются пары запросов, размеченные асессорами.

Данные можно скачать по ссылке: https://drive.google.com/file/d/1n457AAgrMwd5VbT6mGZ_rws3g2wwdEfX/view?usp=sharing

### Метрика качества

Ваш алгоритм должен выдавать пять вариантов ответа для каждого запроса.
Для упрощения задачи метрика качества будет учитывать утилиты и флаги ответа, но не учитывать подставленные значения. Пусть $\{ u_1, \ldots, u_T \}$, $\{ f_1, \ldots, f_T \}$ --- список утилит и множества их флагов ответа алгоритма, $\{v_1, \ldots, v_T \}$, $\{ \phi_1, \ldots, \phi_T \}$ --- список утилит и множества их флагов эталонного ответа. Если ответы отличаются по длине, они дополняются `None` утилитой.

$$ S = \frac{1}{T} \sum_{i=1}^{T} \left(\mathbb{I}[u_i = v_i]\left( 1 + \frac{1}{2}s(f_i, \phi_i)\right) - 1\right)$$

$$ s(f, \phi) = 1 + \frac{2 |f \cap \phi| - |f \cup \phi|}{\max(|f|, |\phi|)} $$

Метрика учитывает, что предсказать правильную утилиту важнее чем правильный флаг. При этом порядок флагов не важен (однако, чтобы корректно

## Предобработка данных (2 балла)

In [82]:
# Скачать архив ветки nlc2cmd
!wget https://github.com/IBM/clai/archive/refs/heads/nlc2cmd.zip

--2024-12-21 19:52:35--  https://github.com/IBM/clai/archive/refs/heads/nlc2cmd.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/IBM/clai/zip/refs/heads/nlc2cmd [following]
--2024-12-21 19:52:35--  https://codeload.github.com/IBM/clai/zip/refs/heads/nlc2cmd
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘nlc2cmd.zip.1’

nlc2cmd.zip.1           [             <=>    ] 103.21M  15.9MB/s    in 6.5s    

2024-12-21 19:52:42 (15.9 MB/s) - ‘nlc2cmd.zip.1’ saved [108220962]



In [83]:
# Разархивировать только папки utils
!unzip nlc2cmd.zip 'clai-nlc2cmd/utils/*' -d ./

# Переместить извлеченные папки в рабочую директорию
!mv clai-nlc2cmd/utils ./utils

# Проверить содержимое папок
!ls ./utils

Archive:  nlc2cmd.zip
2ad172acbed0c1ec870cc39f47635adea39f19c0
   creating: ./clai-nlc2cmd/utils/
   creating: ./clai-nlc2cmd/utils/bashlint/
  inflating: ./clai-nlc2cmd/utils/bashlint/README.md  
  inflating: ./clai-nlc2cmd/utils/bashlint/__init__.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/bash.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/bast.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/bparser.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/butils.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/constants.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/data_tools.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/errors.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/flags.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/grammar.py  
   creating: ./clai-nlc2cmd/utils/bashlint/grammar/
  inflating: ./clai-nlc2cmd/utils/bashlint/grammar/extract_man.py  
  inflating: ./clai-nlc2cmd/utils/bashlint/grammar/grammar100.txt  
  inflating: ./clai-nlc2cmd/utils/bashlint/grammar/top100.t

In [84]:
import sys
PATH_TO_CLAI_UTILS = '/kaggle/working/utils'
sys.path.append(PATH_TO_CLAI_UTILS)

In [85]:
import os

# Путь к файлу, который нужно изменить
file_path = '/kaggle/working/utils/bashlint/butils.py'

# Проверьте, существует ли файл
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        # Читаем содержимое файла
        content = file.read()

    # Заменяем 'collections.MutableSet' на 'collections.abc.MutableSet'
    updated_content = content.replace('collections.MutableSet', 'collections.abc.MutableSet')
    updated_content = updated_content.replace('collections.Mapping', 'collections.abc.Mapping')

    # Записываем изменения обратно в файл
    with open(file_path, 'w') as file:
        file.write(updated_content)

    print(f"Файл {file_path} успешно обновлен.")
else:
    print(f"Файл {file_path} не найден.")


Файл /kaggle/working/utils/bashlint/butils.py успешно обновлен.


In [86]:
import numpy as np
import pandas as pd

from bashlint.data_tools import bash_parser, pretty_print, cmd2template
from metric.metric_utils import compute_metric
from functools import partial

from collections import Counter
import sentencepiece as spm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

import sys
import os

import torch.nn.functional as F
from collections import defaultdict

Считаем данные. В столбце `invocation` находится текстовый запрос, в столбце `cmd` находится релевантная команда.

In [87]:
train_data = pd.read_csv('/kaggle/input/text2bash-data/train.csv')
train_data.head()

,invocation,cmd
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`


В тестовых данных столбец `origin` отвечает за источник данных, значения `handrafted` соответствуют парам, составленными людьми, а `mined` парам, собранным автоматически.

In [88]:
test_data = pd.read_csv('/kaggle/input/text2bash-data/test_data.csv')
test_data.head()

,invocation,cmd,origin
0,create ssh connection to specified ip from spe...,ssh user123@176.0.13.154,handcrafted
1,"search for commands containing string ""zeppeli...",history | grep zeppelin,handcrafted
2,search for location of specified file or appli...,whereis python3,handcrafted
3,grant all rights to root folder,sudo chmod 777 -R /,handcrafted
4,search in running processes for specified name,ps -aux | grep zepp,handcrafted


**Задание**. Проведите предобработку текста. Рекомендуется:
* перевести всё в нижний регистр
* удалить стоп-слова (специфичные для выборки)
* провести стемминг токенов
* удалить все символы кроме латинских букв

In [89]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
train_data['text_cleaned'] = train_data['invocation'].apply(clean_text)
test_data['text_cleaned'] = test_data['invocation'].apply(clean_text)

Для обработки кода воспользуемся функцией `cmd2template`:

In [91]:
train_data['cmd_cleaned'] = train_data['cmd'].apply(partial(cmd2template, loose_constraints=True))
test_data['cmd_cleaned'] = test_data['cmd'].apply(partial(cmd2template, loose_constraints=True))

Разделим данные на обучение и валидацию. Т.к. данных очень мало, то для валидационной выборки выделим только 100 примеров.

In [92]:
valid_data = train_data.iloc[-100:]
train_data = train_data.iloc[:-100]

**Задание**. Стандартный формат входных данных для трансформеров — BPE токены. Воспользуйтесь библиотекой sentencepiece для обучения токенайзеров для текста и кода. Используйте небольшое число токенов.

In [93]:
# Обучение токенизатора для текста
text_spm_path = 'text_tokenizer'
with open('text_train.txt', 'w') as f:
    f.write('\n'.join(train_data['text_cleaned'].tolist()))

spm.SentencePieceTrainer.train(
    f'--input=text_train.txt --model_prefix={text_spm_path} --vocab_size=1000 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3'
)
text_tokenizer = spm.SentencePieceProcessor(model_file=f'{text_spm_path}.model')

# Обучение токенизатора для кода
cmd_spm_path = 'cmd_tokenizer'
with open('cmd_train.txt', 'w') as f:
    f.write('\n'.join(train_data['cmd_cleaned'].tolist()))

spm.SentencePieceTrainer.train(
    f'--input=cmd_train.txt --model_prefix={cmd_spm_path} --vocab_size=500 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3'
)
cmd_tokenizer = spm.SentencePieceProcessor(model_file=f'{cmd_spm_path}.model')

**Задание**. Задайте датасеты и лоадеры для ваших данных. Каждая последовательность должна начинаться с BOS токена и заканчиваться EOS токеном. Рекомендуется ограничить длину входных и выходных последовательностей!

In [94]:
PAD_ID = 0
BOS_ID = 1
EOS_ID = 2


MAX_TEXT_LENGTH = 256
MAX_CODE_LENGTH = 40

BATCH_SIZE = 64

In [95]:
class TextToBashDataset(Dataset):
    def __init__(self, dataframe, text_tokenizer, cmd_tokenizer, max_text_length, max_code_length):
        self.data = dataframe
        self.text_tokenizer = text_tokenizer
        self.cmd_tokenizer = cmd_tokenizer
        self.max_text_length = max_text_length
        self.max_code_length = max_code_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['text_cleaned'].iloc[idx]
        code = self.data['cmd_cleaned'].iloc[idx]

        text_tokens = self.text_tokenizer.encode(text)
        code_tokens = self.cmd_tokenizer.encode(code)

        text_tokens = text_tokens[:self.max_text_length]
        code_tokens = code_tokens[:self.max_code_length - 1]  # Reserve space for EOS

        text_tokens = [BOS_ID] + text_tokens + [EOS_ID]
        code_tokens = [BOS_ID] + code_tokens + [EOS_ID]

        text_padding_len = self.max_text_length + 2 - len(text_tokens)
        code_padding_len = self.max_code_length + 1 - len(code_tokens) # Correction here

        text_tokens += [PAD_ID] * text_padding_len
        code_tokens += [PAD_ID] * code_padding_len

        return {
            'input_ids': torch.tensor(text_tokens),
            'attention_mask': torch.tensor([1 if token != PAD_ID else 0 for token in text_tokens]),
            'decoder_input_ids': torch.tensor(code_tokens[:-1]), # Shifted for teacher forcing
            'decoder_attention_mask': torch.tensor([1 if token != PAD_ID else 0 for token in code_tokens[:-1]]),
            'labels': torch.tensor(code_tokens[1:]) # Target for prediction
        }

In [96]:
train_ds = TextToBashDataset(train_data, text_tokenizer, cmd_tokenizer, MAX_TEXT_LENGTH, MAX_CODE_LENGTH)
valid_ds = TextToBashDataset(valid_data, text_tokenizer, cmd_tokenizer, MAX_TEXT_LENGTH, MAX_CODE_LENGTH)

In [97]:
loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True),
    'valid': DataLoader(valid_ds, batch_size=BATCH_SIZE),
}

## Обучение бейзлайна (2 балла)

In [98]:
from transformers import BertConfig, BertModel, EncoderDecoderConfig, EncoderDecoderModel

**Задание.** Реализуйте модель encoder-decoder ниже. В качестве моделей энкодера и декодера рекомендуется использовать BertModel из библиотеки transformers, заданную через BertConfig. В случае декодера необходимо выставить параметры is_decoder=True и add_cross_attention=True. В качестве модели, <<сцепляющей>> энкодер и декодер, в одну архитектуру рекомендуется использовать EncoderDecoderModel.

**Обратите внимание!** EncoderDecoderModel поддерживает использование кэшированных результатов при последовательной генерации. Это пригодится при реализации beam-search ниже.

Для того, чтобы удобнее задавать модели, рекомендуется реализовать задание модели через конфиг. Ниже представлены базовые параметры, при которых модель должна работать быстро и с приемлемым качеством.

In [99]:
text_model_config = {
    'vocab': text_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

cmd_model_config = {
    'vocab': cmd_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

In [100]:
class TextToBashModel(nn.Module):
    def __init__(self, text_config, cmd_config):
        super().__init__()
        encoder_config = BertConfig(**text_config)
        decoder_config = BertConfig(**cmd_config, is_decoder=True, add_cross_attention=True)
        config = EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config, decoder_config)
        self.model = EncoderDecoderModel(config=config)
        self.model.config.decoder_start_token_id = BOS_ID

    def forward(self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )
        return outputs.logits

**Задание**. Обучите вашу модель ниже.

Рекомендуется:
* в качестве лосса использовать стандартную кросс-энтропию, не забывайте игнорировать PAD токены
* использовать Adam для оптимизации
* не использовать scheduler для бейзлайна (модель легко переобучается с ним)
* использовать early stopping по валидационному лоссу

In [101]:
from torch.optim import AdamW
from tqdm import tqdm

# Инициализация 
model = TextToBashModel(text_model_config, cmd_model_config)
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_ID)
optimizer = AdamW(model.parameters(), lr=1e-4)

# Параметры обучения
num_epochs = 10
patience = 3
best_valid_loss = float('inf')
epochs_no_improve = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Цикл обучения
for epoch in tqdm(range(num_epochs)):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    model.train()
    train_loss = 0.0
    for batch in loaders['train']:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        decoder_input_ids = batch['decoder_input_ids'].to(device)
        decoder_attention_mask = batch['decoder_attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask
        )
        loss = loss_fn(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(loaders['train'])
    print(f"Train Loss: {avg_train_loss:.4f}")

    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for batch in loaders['valid']:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            decoder_input_ids = batch['decoder_input_ids'].to(device)
            decoder_attention_mask = batch['decoder_attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                decoder_attention_mask=decoder_attention_mask
            )
            loss = loss_fn(outputs.view(-1, outputs.size(-1)), labels.view(-1))
            valid_loss += loss.item()

    avg_valid_loss = valid_loss / len(loaders['valid'])
    print(f"Valid Loss: {avg_valid_loss:.4f}")

    # Early stopping
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Сохранение лучшей модели
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print("Early stopping triggered")
            break

print("Finished Training")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10
Train Loss: 5.8689
Valid Loss: 3.8312


 10%|█         | 1/10 [01:56<17:28, 116.52s/it]

Epoch 2/10
Train Loss: 2.6619
Valid Loss: 2.8474


 20%|██        | 2/10 [03:52<15:28, 116.12s/it]

...

Epoch 9/10
Train Loss: 0.9292
Valid Loss: 1.4878


 90%|█████████ | 9/10 [17:23<01:55, 115.91s/it]

Epoch 10/10
Train Loss: 0.8621
Valid Loss: 1.4635


100%|██████████| 10/10 [19:19<00:00, 115.92s/it]

Finished Training


## Генерация команд (2 балла)

**Задание**. Реализуйте алгоритм beam-search в классе BeamSearchGenerator ниже. Ваша реализация должна поддерживать задание температуры софтмакса. Выходы модели, полученные на предыдущих итерациях, необходимо кэшировать для повышения скорости алгоритма. Вместо подсчёта произведения любых вероятностей необходимо считать сумму их логарифмов.

Алгоритм должен возвращать список пар из получившихся выходных последовательностей и логарифмов их вероятностей.

In [105]:
class BeamSearchGenerator:
    def __init__(
            self, pad_id, eos_id, bos_id,
            max_length=20, beam_width=5, temperature=1,
            device='cuda'
    ):
        """
        Parameters
        ----------
        pad_id : int
        eos_id : int
        bos_id : int
        max_length : int
            Maximum length of output sequence
        beam_width : int
            Width of the beam
        temperature : float
            Softmax temperature
        device : torch.device
            Your model device
        """
        self.pad_id = pad_id
        self.eos_id = eos_id
        self.bos_id = bos_id
        self.max_length = max_length
        self.beam_width = beam_width
        self.temperature = temperature
        self.device = device

    def get_result(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        Returns
        -------
        List of tuples: [(sequence, log_prob), ...]
        """
        # Перемещаем входные токены на устройство и добавляем батч-размер 1
        input_text_tokens = input_text_tokens.unsqueeze(0).to(self.device)  # Shape: [1, seq_len]
        attention_mask = (input_text_tokens != self.pad_id).long()

        # Получаем выходы энкодера и кэшируем их
        encoder_outputs = model.model.encoder(
            input_ids=input_text_tokens,
            attention_mask=attention_mask,
            return_dict=True
        )

        # Инициализируем луч с начальной последовательностью [BOS_ID] и лог вероятностью 0
        beams = [([self.bos_id], 0.0)]
        # Список завершенных последовательностей
        completed_beams = []

        for _ in range(self.max_length):
            # Собираем все текущие активные последовательности
            active_beams = [beam for beam in beams if beam[0][-1] != self.eos_id]
            # Если нет активных последовательностей, прекращаем генерацию
            if len(active_beams) == 0:
                break

            # Подготовка декодерских входов: батч всех активных последовательностей
            current_batch_size = len(active_beams)
            decoder_input_ids = torch.tensor([beam[0] for beam in active_beams], device=self.device)  # Shape: [batch, seq_len]
            decoder_attention_mask = (decoder_input_ids != self.pad_id).long()

            # Получаем выходы декодера
            decoder_outputs = model.model.decoder(
                input_ids=decoder_input_ids,
                attention_mask=decoder_attention_mask,
                encoder_hidden_states=encoder_outputs.last_hidden_state,
                encoder_attention_mask=attention_mask,
                return_dict=True
            )

            # Получаем логиты для последнего токена в каждой последовательности
            logits = decoder_outputs.logits[:, -1, :]  # Shape: [batch, vocab_size]
            logits = logits / self.temperature
            log_probs = F.log_softmax(logits, dim=-1)  # Shape: [batch, vocab_size]

            # Выбираем топ beam_width вероятностей и их индексы для каждой последовательности
            top_log_probs, top_indices = torch.topk(log_probs, self.beam_width, dim=-1)  # Each of shape [batch, beam_width]

            # Расширяем текущие лучи с топ кандидатов
            all_candidates = []
            for i in range(current_batch_size):
                seq, seq_log_prob = active_beams[i]
                for j in range(self.beam_width):
                    token = top_indices[i, j].item()
                    token_log_prob = top_log_probs[i, j].item()
                    new_seq = seq + [token]
                    new_log_prob = seq_log_prob + token_log_prob
                    if token == self.eos_id:
                        completed_beams.append((new_seq, new_log_prob))
                    else:
                        all_candidates.append((new_seq, new_log_prob))

            # Если нет кандидатов, прекращаем генерацию
            if len(all_candidates) == 0:
                break

            # Сортируем все кандидаты по log_prob и выбираем top beam_width
            all_candidates = sorted(all_candidates, key=lambda x: x[1], reverse=True)
            beams = all_candidates[:self.beam_width]

        # Добавляем оставшиеся лучи, которые могли быть завершены EOS на последнем шаге
        completed_beams.extend(beams)

        # Сортируем все завершенные лучи и выбираем top beam_width
        completed_beams = sorted(completed_beams, key=lambda x: x[1], reverse=True)
        return completed_beams[:self.beam_width]

Протестируйте на нескольких примерах работу вашего алгоритма. Если всё реализовано правильно, то как минимум на трёх примерах из 5 всё должно работать правильно.

In [106]:
beam_search_engine = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=10,
    temperature=0.8, device='cuda',
)

In [108]:
with torch.no_grad():
    for i in range(5):
        print()
        print('text:', valid_data.invocation.iloc[i])
        print('true:', valid_data.cmd.iloc[i])
        print('true cleaned:', valid_data.cmd_cleaned.iloc[i])

        input_text = valid_data.text_cleaned.iloc[i]
        input_tokens = torch.tensor([BOS_ID] + text_tokenizer.encode(input_text) + [EOS_ID]).to(device)
        pred = beam_search_engine.get_result(model, input_tokens)

        scores = []
        for tokens, proba in pred:
            # Прямое декодирование списка токенов
            pred_cmd = cmd_tokenizer.decode(tokens)
            score = compute_metric(pred_cmd, 1, valid_data.cmd.iloc[i])
            scores.append(score)
            print(pred_cmd, proba)
        if scores:
            print(f"Best score: {max(scores)}")
        else:
            print("No predictions generated.")


text: searches through the root filesystem ("/") for the file named chapter1, and prints the location
true: find / -name Chapter1 -type f -print
true cleaned: find Path -name Regex -type f -print
find Path -name Regex -print -1.4612334616249427
find Path -name Regex -type f -1.8078021722612903
find Path -name Regex -exec echo {} \; -2.8884599677985534
find Path -name Regex -type f -print -2.9969136434374377
find Path -name Regex -3.0307849295204505
find Path -name Regex -print0 -3.5620478339260444
find Path -name Regex -type f -print0 -3.5942550020990893
find Path -type f -name Regex -print0 -4.138198224420194
find Path -name Regex -printf "%f\n" -4.789454309619032
find Path -name Regex -name Regex -4.8851425586035475
Best score: 1.0

text: searches through the root filesystem ("/") for the file named chapter1.
true: find / -name Chapter1 -type f
true cleaned: find Path -name Regex -type f
find Path -name Regex -type f -1.109224942571018
find Path -name Regex -2.074280857166741
find P

**Задание**. Дополните функцию для подсчёта качества. Посчитайте качество вашей модели на валидационном и тестовых датасетов.

In [111]:
def compute_all_scores(model, df, beam_engine):
    all_scores = []
    print("Количество итераций", df.text_cleaned.values.shape)
    for i, (text, target_cmd) in tqdm(enumerate(zip(df.text_cleaned.values, df.cmd.values))):
        input_tokens = torch.tensor([BOS_ID] + text_tokenizer.encode(text) + [EOS_ID]).to(device)
        predictions = beam_engine.get_result(model, input_tokens)

        # Get only 5 top results
        predictions = predictions[:5]
        object_scores = []
        for output_tokens, proba in predictions:
            output_cmd = cmd_tokenizer.decode(output_tokens)
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)

        if object_scores:
            all_scores.append(max(object_scores))
        else:
            all_scores.append(0.0) # Assign a score of 0 if no predictions

    return all_scores

Ваша цель при помощи подбора параметров модели и генерации получить средний скор на валидации >= 0.2, скор `handcrafted` части теста >= 0.1. На `mined` части датасета скор может быть низкий, т.к. некоторых команд из датасета нет в обучении.

**Обратите внимание.** Так как датасет для обучения не очень большой, а данные достаточно нестабильные, подбор параметров может очень сильно влиять на модель. Некоторые полезные советы:
* Отслеживайте качество модели после каждой эпохи, не забывайте про early stopping
* Вы можете сразу приступить к следующей части. Побитие скора в этой части задания при помощи трюков из бонусной части считается валидным.

In [112]:
# Calculate scores on validation set
valid_scores = compute_all_scores(model, valid_data, beam_search_engine)
print(f"Average validation score: {np.mean(valid_scores)}")

# Calculate scores on test set
test_scores = compute_all_scores(model, test_data, beam_search_engine)
print(f"Average test score: {np.mean(test_scores)}")

# Calculate scores on handcrafted test data
handcrafted_test_data = test_data[test_data['origin'] == 'handcrafted']
handcrafted_scores = compute_all_scores(model, handcrafted_test_data, beam_search_engine)
print(f"Average handcrafted test score: {np.mean(handcrafted_scores)}")

# Calculate scores on mined test data
mined_test_data = test_data[test_data['origin'] == 'mined']
mined_scores = compute_all_scores(model, mined_test_data, beam_search_engine)
print(f"Average mined test score: {np.mean(mined_scores)}")

Количество итераций (100,)


100it [01:41,  1.02s/it]


Average validation score: 0.22001388888888887
Количество итераций (721,)


721it [12:13,  1.02s/it]


Average test score: -0.19282398234374654
Количество итераций (129,)


129it [02:11,  1.02s/it]


Average handcrafted test score: 0.14664082687338503
Количество итераций (592,)


592it [10:01,  1.02s/it]

Average mined test score: -0.2667951992170742


## Улучшение модели (4 балла)

Вы реализовали бейзлайн, пришло время улучшить качество модели. Т.к. это последнее задание, мы не будем предлагать конкретные шаги, а только дадим несколько советов.

1. Большой источник информации о работе командной строке — её документация, man. Один из способов улучшения модели - использование мана для генерации новых примеров. Структурированный ман можно найти по ссылке https://github.com/IBM/clai/blob/nlc2cmd/docs/manpage-data.md.
2. Ещё один способ улучшить модель, разделить предсказание утилит и флагов. Т.к. задача предсказания утилит более важная, вы можете натренировать модель, которая предсказывает последовательность утилит, а затем к каждой утилите генерировать флаги.
3. Можно аугментировать данные, чтобы увеличить выборку.
4. Можно в качество входа подавать не только текстовый запрос, но и описание из мана. Т.к. всё описание достаточно большое, нужно сделать дополнительную модель, которая будет выбирать команды, для которых нужно вытащить описание.
5. Найти дополнительные данные, улучшающие обучение
6. Как всегда можно просто сделать больше слоёв, увеличить размер скрытого слоя и т.д.

От вас ожидается скор на валидации >= 0.25, `mined` >= 0, `handrafted` >= 0.15.

## Бонусные баллы (до 3 баллов)

При существенном улучшении качества будут назначаться бонусные баллы. На тестовых датасетах реально выбить качество >= 0.3 на каждом, но усилий потребуется немало...